In [1]:
import ROOT
from matplotlib import pyplot as plt
import pandas as pd

Welcome to JupyROOT 6.28/04


In [3]:
file_name = "rechit_histograms_20231031.root"
my_file = ROOT.TFile.Open(file_name)

In [5]:
#open file and read in histograms
my_file.cd()
my_file.ls()
my_file.goodTracks.cd()
histograms = {}
for key in my_file.GetListOfKeys():
    hist = key.ReadObj()
    histograms[hist.GetName()] = hist

TFile**		rechit_histograms_20231031.root	
 TFile*		rechit_histograms_20231031.root	
  TDirectoryFile*		goodTracks	goodTracks
   KEY: TH2D	XY;1	XY
   KEY: TH2D	RZ;1	RZ
   KEY: TH1I	nHitsPerLayer;1	nHitsPerLayer
   KEY: TH1I	nTracksPerLayer;1	nTracksPerLayer
   KEY: TH1I	nTracksPerPair;1	nTracksPerPair
   KEY: TH1I	nTracksPerTriple;1	nTracksPerTriple
   KEY: TH2I	nTracks;1	nTracks
   KEY: TH2I	nTracksSeq;1	nTracksSeq
   KEY: TH1I	nHitsPerTrack;1	nHitsPerTrack
   KEY: TH1I	nSegmentsPerTrack;1	nSegmentsPerTrack
   KEY: TH1I	nSegmentsNoLoopPerTrack;1	nSegmentsNoLoopPerTrack
   KEY: TDirectoryFile	maxHitDistance;1	maxHitDistance
   KEY: TDirectoryFile	nHitsPerTrackSegment;1	nHitsPerTrackSegment
   KEY: TDirectoryFile	pt;1	pt
   KEY: TDirectoryFile	tof;1	tof
   KEY: TDirectoryFile	iphi;1	iphi
   KEY: TDirectoryFile	z0;1	z0
   KEY: TDirectoryFile	curvature;1	curvature
   KEY: TDirectoryFile	dca;1	dca
   KEY: TDirectoryFile	dcaRc;1	dcaRc
   KEY: TDirectoryFile	rc;1	rc
   KEY: TDirectoryFile	the